**[MDC-01]** 필요한 모듈을 임포트하고 난수의 시드를 설정한다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline
np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDC-02]** MNIST 데이터 세트를 준비한다.

In [3]:
mnist = input_data.read_data_sets("./토요일_src/data/", one_hot=True)

Extracting ./토요일_src/data/train-images-idx3-ubyte.gz
Extracting ./토요일_src/data/train-labels-idx1-ubyte.gz
Extracting ./토요일_src/data/t10k-images-idx3-ubyte.gz
Extracting ./토요일_src/data/t10k-labels-idx1-ubyte.gz


**[MDC-03]** 필터에 해당하는 Variable을 준비하고 입력 데이터에 필터와 풀링 계층을 적용하는 계산식을 정의한다.

In [4]:
num_filters = 16

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters],
                                         stddev=0.1))
h_conv = tf.nn.conv2d(x_image, W_conv,
                      strides=[1,1,1,1], padding='SAME')
h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                       strides=[1,2,2,1], padding='SAME')

**[MDC-04]** 풀링 계층의 출력을 전 결합층을 경유해서 소프트맥스 함수로 입력하는 계산식을 정의한다.

In [5]:
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

num_units1 = 14*14*num_filters
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

**[MDC-05]** 오차 함수 loss, 트레이닝 알고리즘 train_step, 정답률 accuracy를 정의한다.

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[MDC-06]** 세션을 준비하고 Variable을 초기화한다.

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

**[MDC-07]** 파라미터 최적화를 4000회 반복한다

최종적으로 테스트 세트에 대해 약 98%의 정답률을 얻을 수 있다.

In [8]:
i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, './토요일_src/data/ckpt/mdc_session.ckpt', global_step=i)

Step: 100, Loss: 2387.759521, Accuracy: 0.928200
Step: 200, Loss: 2044.267822, Accuracy: 0.937100
Step: 300, Loss: 1424.071777, Accuracy: 0.958200
Step: 400, Loss: 1163.767334, Accuracy: 0.963600
Step: 500, Loss: 1061.417480, Accuracy: 0.969400
Step: 600, Loss: 1003.050232, Accuracy: 0.968700
Step: 700, Loss: 1050.239380, Accuracy: 0.969400
Step: 800, Loss: 870.526978, Accuracy: 0.974300
Step: 900, Loss: 846.846191, Accuracy: 0.974000
Step: 1000, Loss: 900.901489, Accuracy: 0.972500
Step: 1100, Loss: 826.629150, Accuracy: 0.974800
Step: 1200, Loss: 882.477478, Accuracy: 0.972700
Step: 1300, Loss: 1023.264709, Accuracy: 0.971500
Step: 1400, Loss: 784.716675, Accuracy: 0.976400
Step: 1500, Loss: 830.614868, Accuracy: 0.975300
Step: 1600, Loss: 724.573486, Accuracy: 0.976900
Step: 1700, Loss: 691.795898, Accuracy: 0.979500
Step: 1800, Loss: 717.886353, Accuracy: 0.977500
Step: 1900, Loss: 669.335083, Accuracy: 0.979500
Step: 2000, Loss: 729.064453, Accuracy: 0.978200


**[MDC-08]** 세션 정보를 저장한 파일이 생성되고 있음을 확인한다.

In [9]:
!dir ./토요일_src/data/ckpt/mdc_session*

./토요일_src/data/ckpt/mdc_session.ckpt-1600.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-1600.index
./토요일_src/data/ckpt/mdc_session.ckpt-1600.meta
./토요일_src/data/ckpt/mdc_session.ckpt-1700.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-1700.index
./토요일_src/data/ckpt/mdc_session.ckpt-1700.meta
./토요일_src/data/ckpt/mdc_session.ckpt-1800.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-1800.index
./토요일_src/data/ckpt/mdc_session.ckpt-1800.meta
./토요일_src/data/ckpt/mdc_session.ckpt-1900.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-1900.index
./토요일_src/data/ckpt/mdc_session.ckpt-1900.meta
./토요일_src/data/ckpt/mdc_session.ckpt-2000.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-2000.index
./토요일_src/data/ckpt/mdc_session.ckpt-2000.meta
./토요일_src/data/ckpt/mdc_session.ckpt-3600.data-00000-of-00001
./토요일_src/data/ckpt/mdc_session.ckpt-3600.index
./토요일_src/data/ckpt/mdc_session.ckpt-3600.meta
./토요일_src/data/ckpt/mdc_session.ckpt-3700.data-00000-of-00

In [10]:
sess.close()